In [ ]:
# Import necessary libraries
import numpy as np
from mpi4py import MPI
from dolfinx import mesh, fem, nls, io
from dolfinx.fem.petsc import NonlinearProblem
from dolfinx.nls.petsc import NewtonSolver
from dolfinx.fem import locate_dofs_geometrical
from petsc4py.PETSc import ScalarType
from petsc4py import PETSc
from ufl import TrialFunction, TestFunction, div, dx
from ufl.finiteelement import FiniteElement, MixedElement
import ufl
import basix
from dolfinx.plot import vtk_mesh
import pyvista as pv
import os
from dolfinx.fem import assemble_scalar, form  
from ufl import dot, FacetNormal, Measure  
from dolfinx.mesh import exterior_facet_indices

# Create directory for storing simulation frames
os.makedirs("frames", exist_ok=True)

# Simulation parameters
Lx, Ly = 10.0, 10.0  # Domain dimensions
nx, ny = 5, 5        # Number of elements in x and y directions
g = 9.81             # Gravitational acceleration
n = 0.035            # Manning's roughness coefficient
t, dt, T = 0.0, 0.1, 9000.0  # Time-related parameters
num_steps = int(T / dt)       # Total number of time steps

# Create a rectangular mesh
domain = mesh.create_rectangle(
    MPI.COMM_WORLD, [np.array([0.0, 0.0]), np.array([Lx, Ly])], [nx, ny], cell_type=mesh.CellType.triangle
)
domain.topology.create_connectivity(domain.topology.dim - 1, domain.topology.dim)
x = domain.geometry.x  # Mesh geometry

# Define a mixed finite element space for water depth and discharges
element = basix.ufl.mixed_element([basix.ufl.element("CG", str(domain.ufl_cell()), 1)] * 3)
V = fem.functionspace(domain, element)

# Extract individual subspaces and mapping for each field (h, qx, qy)
num_subs = V.num_sub_spaces
spaces = []
maps = []
for i in range(num_subs):
    space_i, map_i = V.sub(i).collapse()
    spaces.append(space_i)
    maps.append(map_i)

# Compute flux across boundaries
def compute_flux(h, qx, qy):
    n = FacetNormal(domain)  # Unit normal vector on facets
    q = ufl.as_vector([qx, qy])  # Discharge vector
    flux_integrand = dot(q, n)  # Flux contribution
    exterior_facet_measure = Measure("exterior_facet", domain=domain)
    flux_integral = assemble_scalar(form(flux_integrand * exterior_facet_measure))  # Total flux
    inflow_form = ufl.conditional(flux_integrand < 0, -flux_integrand, 0) * exterior_facet_measure  # Inflow
    outflow_form = ufl.conditional(flux_integrand > 0, flux_integrand, 0) * exterior_facet_measure  # Outflow
    inflow = assemble_scalar(form(inflow_form))
    outflow = assemble_scalar(form(outflow_form))
    return inflow, outflow

# Define functions for current and previous time steps
W = fem.Function(V)  
h, qx, qy = W.split()  # Split mixed function into components
W_prev = fem.Function(V)
h_prev, qx_prev, qy_prev = W_prev.split()

# Initialize water depth and discharges
h.interpolate(lambda x: np.full_like(x[0], 4.5))
qx.interpolate(lambda x: np.full_like(x[0], 0.0))
qy.interpolate(lambda x: np.full_like(x[0], 0.0))
h_prev.interpolate(lambda x: np.full_like(x[0], 4.5))
qx_prev.interpolate(lambda x: np.full_like(x[0], 0.0))
qy_prev.interpolate(lambda x: np.full_like(x[0], 0.0))

# Define trial and test functions
U = TrialFunction(V)
V_test = TestFunction(V)

# Define the weak form of the equations
def weak_form(U, U_prev, V_test):
    h, qx, qy = U
    h_prev, qx_prev, qy_prev = U_prev
    v_h, v_qx, v_qy = V_test
    continuity = (h - h_prev) / dt * v_h * dx + (qx.dx(0) * v_h + qy.dx(1) * v_h) * dx
    momentum_x = (qx - qx_prev) / dt * v_qx * dx + g * h * h.dx(0) * v_qx * dx + g * n**2 * ((abs(qx) * qx) / h**(7/3)) * v_qx * dx
    momentum_y = (qy - qy_prev) / dt * v_qy * dx + g * h * h.dx(1) * v_qy * dx + g * n**2 * ((abs(qy) * qy) / h**(7/3)) * v_qy * dx
    return continuity + momentum_x + momentum_y

# Locate boundary facets and apply boundary conditions
def boundary_marker(x):
    left_boundary_part = np.isclose(x[0], 0) & (x[1] >= 0) & (x[1] < 2000)  
    upper_boundary_part = np.isclose(x[1], 0) & (x[0] > 0) & (x[0] < 1500)  
    return left_boundary_part | upper_boundary_part

boundary_facets = mesh.locate_entities_boundary(domain, domain.topology.dim - 1, boundary_marker)
# Apply Dirichlet boundary conditions for each field
sub_dofs_h = fem.locate_dofs_topological(V.sub(0), domain.topology.dim - 1, boundary_facets)
sub_dofs_qx = fem.locate_dofs_topological(V.sub(1), domain.topology.dim - 1, boundary_facets)
sub_dofs_qy = fem.locate_dofs_topological(V.sub(2), domain.topology.dim - 1, boundary_facets)

# Create boundary conditions
bc_h = fem.dirichletbc(fem.Constant(domain, ScalarType(4.51)), sub_dofs_h, V.sub(0))
bc_qx = fem.dirichletbc(fem.Constant(domain, ScalarType(1.0)), sub_dofs_qx, V.sub(1))
bc_qy = fem.dirichletbc(fem.Constant(domain, ScalarType(1.0)), sub_dofs_qy, V.sub(2))
bcs = [bc_h, bc_qx, bc_qy]

# Calculate mass conservation residual for each node/cell
def mass_conservation_residual(h, qx, qy, h_prev, qx_prev, qy_prev, dx, dy, dt, maps):
    # Extract values using the maps
    h_values = h.x.array[maps[0]]
    qx_values = qx.x.array[maps[1]]
    qy_values = qy.x.array[maps[2]]
    h_prev_values = h_prev.x.array[maps[0]]
    qx_prev_values = qx_prev.x.array[maps[1]]
    qy_prev_values = qy_prev.x.array[maps[2]]
    
    # Initialize residual array
    residual = np.zeros_like(h_values)
    
    # Time derivative term for h
    residual += (h_values - h_prev_values) / dt
    
    # Spatial derivatives for qx and qy (using finite differences)
    residual += np.gradient(qx_values, dx)  # the derivative method for qx
    residual += np.gradient(qy_values, dy)  # the derivative method for qy
    return residual

# Define the nonlinear problem for SWE
class SWEProblem(NonlinearProblem):
    def __init__(self, weak_form, W, bcs):
        super().__init__(weak_form, W)
        self.weak_form = weak_form  
        self.W = W                  
        self.bcs = bcs              
    def F(self, snes, x, residual):
        self.W.x.array[:] = x.array_r
        self.W.x.scatter_forward()
        fem.petsc.assemble_vector(residual, fem.form(self.weak_form))
        fem.petsc.apply_lifting(residual, [fem.form(self.weak_form)], [self.bcs])
        residual.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE)
        fem.petsc.set_bc(residual, self.bcs)
    def J(self, x, jacobian):
        a = ufl.derivative(self.weak_form, self.W)
        fem.petsc.assemble_matrix(jacobian, fem.form(a), self.bcs)
        jacobian.assemble()

# Initialize problem and solve using Newton's method
weak_form_expr = weak_form(W, W_prev, V_test)
problem = SWEProblem(weak_form_expr, W, bcs)
snes = PETSc.SNES().create(MPI.COMM_WORLD)
from dolfinx.fem.petsc import create_matrix, create_vector
snes.setFunction(problem.F, create_vector(fem.form(weak_form_expr)))
snes.setJacobian(problem.J, create_matrix(fem.form(ufl.derivative(weak_form_expr, W))))
snes.setType(PETSc.SNES.Type.NEWTONLS)  
snes.setTolerances(rtol=1e-6, atol=1e-6, max_it=500)

# Main time-stepping loop
for step in range(num_steps + 1):
    t = step * dt
    # Compute mass conservation residual (optional for diagnostics)
    residual = mass_conservation_residual(h, qx, qy, h_prev, qx_prev, qy_prev, Lx / nx, Ly / ny, dt, maps)
    print(f"Step {step}, Mass Conservation Residual: {np.linalg.norm(residual)}")
    
    # Solve the nonlinear problem
    snes.solve(None, W.x.petsc_vec)
    W.x.scatter_forward()
    
    # Apply constraints on water depth and discharges
    h_values = h.x.array[maps[0]]
    h_values[h_values < 4.5] = 4.5
    h_values[h_values > 20.0] = 20.0
    h.x.array[maps[0]] = h_values
    qx_values = qx.x.array[maps[1]]
    qx_values[qx_values < 0.0] = 0.0
    qx.x.array[maps[1]] = qx_values
    qy_values = qy.x.array[maps[2]]
    qy_values[qy_values < 0.0] = 0.0
    qy.x.array[maps[2]] = qy_values
    
    # Update previous time step values
    h_prev.x.array[:] = h.x.array[:]
    qx_prev.x.array[:] = qx.x.array[:]
    qy_prev.x.array[:] = qy.x.array[:]
